In [ ]:
# Importação das bibliotecas
import numpy as np 
import pandas as pd 
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from scipy import spatial
import time

In [ ]:
# Importação dos dados
# As linhas de código abaixo devem ser descomentadas quando os arquivos necessários não estiverem no diretório "Arquivos" do Google Colaboratory 

#from google.colab import files
#uploaded = files.upload()

Saving movies_new.csv to movies_new.csv
Saving ratings.csv to ratings.csv
Saving users_new.csv to users_new.csv


In [ ]:
ratings = pd.read_csv(r"ratings.csv",
        sep=r'\s*;\s*',
        engine='python')

movies_new = pd.read_csv(r"movies_new.csv",
        sep=r',',
        engine='python')

users_new = pd.read_csv(r"users_new.csv",
        sep=r',',
        engine='python')

In [ ]:
ratings

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [ ]:
movies_new

,MovieID,Title,Drama,Animation,Musical,Action,Comedy,Adventure,Romance,Children's,Thriller,Crime,Western,Documentary,Mystery,Horror,Sci-Fi,Film-Noir,War,Fantasy
0,1,Toy Story (1995),0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3,Grumpier Old Men (1995),0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,3948,Meet the Parents (2000),0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3879,3949,Requiem for a Dream (2000),1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3880,3950,Tigerland (2000),1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3881,3951,Two Family House (2000),1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
users_new

,UserID,Gender,Age,Occupation 0,Occupation 1,Occupation 2,Occupation 3,Occupation 4,Occupation 5,Occupation 6,Occupation 7,Occupation 8,Occupation 9,Occupation 10,Occupation 11,Occupation 12,Occupation 13,Occupation 14,Occupation 15,Occupation 16,Occupation 17,Occupation 18,Occupation 19,Occupation 20
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,3,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,4,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,6036,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6036,6037,0.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,6038,0.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6038,6039,0.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#6040 usuários = 6040 linhas
#21 colunas de ocupação + idade + gênero + 3883 colunas de filmes = 3906 colunas
#colunas = Filme 1	Filme 2	.....	Filme 3883,	Idade,	Gênero,	Ocupação 0,	Ocupação 1,	.....	Ocupação 20
matrix_cos_users = np.zeros((6040,3906))

# Criação de um dicionário que codifica o ID de um filme para um valor entre 0 e 3882, de acordo com movies_new
movies_index_dict = {}
for i in range(3883):
  movies_index_dict[movies_new['MovieID'][i]] = i

In [ ]:
#Preenchendo informações do ratings
for i in range (1000209):
    row_index = ratings['UserID'][i] - 1
    column_index = movies_index_dict[ratings['MovieID'][i]]
    matrix_cos_users[row_index,column_index] = ratings['Rating'][i]

#Preenchendo informações do usuário
for i in range(6040):
  matrix_cos_users[i][3905] = users_new["Occupation 20"][i]
  matrix_cos_users[i][3904] = users_new["Occupation 19"][i]
  matrix_cos_users[i][3903] = users_new["Occupation 18"][i]
  matrix_cos_users[i][3902] = users_new["Occupation 17"][i]
  matrix_cos_users[i][3901] = users_new["Occupation 16"][i]
  matrix_cos_users[i][3900] = users_new["Occupation 15"][i]
  matrix_cos_users[i][3899] = users_new["Occupation 14"][i]
  matrix_cos_users[i][3898] = users_new["Occupation 13"][i]
  matrix_cos_users[i][3897] = users_new["Occupation 12"][i]
  matrix_cos_users[i][3896] = users_new["Occupation 11"][i]
  matrix_cos_users[i][3895] = users_new["Occupation 10"][i]
  matrix_cos_users[i][3894] = users_new["Occupation 9"][i]
  matrix_cos_users[i][3893] = users_new["Occupation 8"][i]
  matrix_cos_users[i][3892] = users_new["Occupation 7"][i]
  matrix_cos_users[i][3891] = users_new["Occupation 6"][i]
  matrix_cos_users[i][3890] = users_new["Occupation 5"][i]
  matrix_cos_users[i][3889] = users_new["Occupation 4"][i]
  matrix_cos_users[i][3888] = users_new["Occupation 3"][i]
  matrix_cos_users[i][3887] = users_new["Occupation 2"][i]
  matrix_cos_users[i][3886] = users_new["Occupation 1"][i]
  matrix_cos_users[i][3885] = users_new["Occupation 0"][i]
  matrix_cos_users[i][3884] = users_new["Age"][i]
  matrix_cos_users[i][3883] = users_new["Gender"][i]

In [ ]:
#3883 filmes = 3883 linhas
#6040 usuários + 18 gêneros cinematográficos = 6058 colunas
#colunas = Usuario1,Usuario2,...Usuario6040, Genero1,Genero2,...Genero18
matrix_cos_movies = np.zeros((3883,6058))

In [ ]:
#Preenchendo informações do ratings
for i in range (1000209):
    row_index = movies_index_dict[ratings['MovieID'][i]]
    column_index = ratings['UserID'][i] - 1
    matrix_cos_movies[row_index,column_index] = ratings['Rating'][i]

#Preenchendo informações do filme
for i in range(3883):
  matrix_cos_movies[i][6057] = movies_new["Fantasy"][i]
  matrix_cos_movies[i][6056] = movies_new["War"][i]
  matrix_cos_movies[i][6055] = movies_new["Film-Noir"][i]
  matrix_cos_movies[i][6054] = movies_new["Sci-Fi"][i]
  matrix_cos_movies[i][6053] = movies_new["Horror"][i]
  matrix_cos_movies[i][6052] = movies_new["Mystery"][i]
  matrix_cos_movies[i][6051] = movies_new["Documentary"][i]
  matrix_cos_movies[i][6050] = movies_new["Western"][i]
  matrix_cos_movies[i][6049] = movies_new["Crime"][i]
  matrix_cos_movies[i][6048] = movies_new["Thriller"][i]
  matrix_cos_movies[i][6047] = movies_new["Children's"][i]
  matrix_cos_movies[i][6046] = movies_new["Romance"][i]
  matrix_cos_movies[i][6045] = movies_new["Adventure"][i]
  matrix_cos_movies[i][6044] = movies_new["Comedy"][i]
  matrix_cos_movies[i][6043] = movies_new["Action"][i]
  matrix_cos_movies[i][6042] = movies_new["Musical"][i]
  matrix_cos_movies[i][6041] = movies_new["Animation"][i]
  matrix_cos_movies[i][6040] = movies_new["Drama"][i]

In [ ]:
# Criação das matrizes de similaridade
sim_users = np.zeros((6040,6040))
sim_movies = np.zeros((3883,3883))

# Criação das matrizes de estimativas
estimated_ratings_users = np.zeros((6040,3883))
estimated_ratings_movies = np.zeros((3883,6040))

In [ ]:
# Criação das funções auxiliares

def similarity(X, Y, type):
  if (np.dot(X,X) <= 1e-10 or np.dot(Y,Y) <= 1e-10):
    return 0
  if (type == 'cosine'):
    return (1 - spatial.distance.cosine(X,Y))
  elif (type == 'euclidean'):
    temp = spatial.distance.euclidean(X,Y)
    if (temp == 0):
      return 0
    else:
      return (1/temp)

def estimated_rating(sim_vec, ratings_column, k):
  sim_index_dict = {}
  sim_list = []
  n_items = len(sim_vec)
  
  for i in range(n_items):
    if (ratings_column[i] != 0):
      sim_list.append(sim_vec[i])
      sim_index_dict[sim_vec[i]] = i

  # Ordenação da lista com as maiores similaridades
  sim_list.sort(reverse=True)

  # Caso não hajam avaliações em ratings_column, a estimativa não é calculada
  if (len(sim_list) == 0):
    return 0.0

  sim_best_list = []
  # Verificando se a quantidade de similaridades correspondentes às avaliações em ratings_column é maior ou menor que k
  if (len(sim_list) >= k):
    for i in range(k):
      sim_best_list.append(sim_list[i])
  else:
    for i in range(len(sim_list)):
      sim_best_list.append(sim_list[i])

  # A estimativa não é calculada caso haja menos que 10 similaridades correspondentes às avaliações em ratings_column
  if (len(sim_best_list) < 10):
    return 0

  mean_rating = 0
  for i in range(len(sim_best_list)):
    mean_rating = mean_rating + sim_best_list[i]*ratings_column[sim_index_dict[sim_best_list[i]]]

  # Caso a soma das maiores similaridades seja muito pequena, a estimativa também não é calculada
  if (sum(sim_best_list) > 0.0001):
    return mean_rating/sum(sim_best_list)
  else:
    return 0

def mean_squared_error(X_target, X_estimated):
  n_rows = 0
  n_columns = 0
  n_total = 0
  mse = 0

  if (len(X_target.shape) == 2):
    n_rows = X_target.shape[0]
    n_columns = X_target.shape[1]
  else:
    n_rows = X_target.shape[0]

  if (len(X_target.shape) == 2):
    for i in range(n_rows):
      for j in range(n_columns):
        # Um elemento nulo da matriz X_target significa que não há avaliação
        if (X_target[i][j] != 0):
          n_total += 1
          mse = mse + (X_target[i][j] - X_estimated[i][j])*(X_target[i][j] - X_estimated[i][j])
  else:
    for i in range(n_rows):
      if (X_target[i] != 0):
        n_total += 1
        mse = mse + (X_target[i] - X_estimated[i])*(X_target[i] - X_estimated[i])

  return mse/n_total

In [ ]:
# Criação de um vetor com os índices referentes a 20 usuários aleatórios
random_index = np.random.randint(low=0,high=6040,size=20)

In [ ]:
# Criação dos dicionários com os resultados de validação
# Estrutura dos elementos dos dicionários -> dict[mse_score] = [sim_type, k]
mse_users_dict_1 = {}      # Teste do algoritmo baseado em usuários, considerando as ocupações
mse_users_dict_2 = {}      # Teste do algoritmo baseado em usuários sem as ocupações
mse_users_dict_3 = {}      # Teste do algoritmo baseado em usuários, considerando somente as avaliações
mse_movies_dict_1 = {}     # Teste do algoritmo baseado em filmes, considerando os gêneros
mse_movies_dict_2 = {}     # Teste do algoritmo baseado em filmes, considerando somente as avaliações

sim_type = ['cosine','euclidean']                                       # Lista com os tipos de similaridade
k_values = np.linspace(start=36,stop=45,num=10,dtype=int)               # Lista com todos os valores de k

In [ ]:
# Validação para método baseado em usuários, considerando as ocupações
start_time = time.time()
for sim in sim_type:
  for k in k_values:
    mean_mse = 0
    for i in random_index:
      # Cálculo de todas as similaridades para o usuário i
      for j in range(6040):
        if (j != i):
          sim_users[i][j] = similarity(X=matrix_cos_users[i],Y=matrix_cos_users[j],type=sim)
        else:
          sim_users[i][j] = 0
      
      evaluated_index = []
      for m in range(3883):
        if (matrix_cos_users[i][m] != 0):
          evaluated_index.append(m)      

      X_target = []
      X_estimated = []
      # Cálculo das estimativas de nota para o usuário i
      for j in evaluated_index:
        estimated_ratings_users[i][j] = estimated_rating(sim_vec=sim_users[i],ratings_column=matrix_cos_users[:,j],k=k)
        if (estimated_ratings_users[i][j] > 0):
          X_target.append(matrix_cos_users[i][j])
          X_estimated.append(estimated_ratings_users[i][j])

      X_target = np.array(X_target)
      X_estimated = np.array(X_estimated)
      mean_mse += mean_squared_error(X_target=X_target,X_estimated=X_estimated)
    mean_mse = mean_mse/len(random_index)
    mse_users_dict_1[mean_mse] = [sim,k]

print('Tempo de execução:',time.time()-start_time,'segundos')
print('Melhores hiperparâmetros:',mse_users_dict_1[min(list(mse_users_dict_1.keys()))],'-> MSE:',min(list(mse_users_dict_1.keys())))

Tempo de execução: 296.7809057235718 segundos
Melhores hiperparâmetros: ['cosine', 45] -> MSE: 0.8687271538695738


In [ ]:
# Limpeza das matrizes de similaridade e de estimativas
for i in range(sim_users.shape[0]):
  for j in range(sim_users.shape[1]):
    sim_users[i][j] = 0

for i in range(estimated_ratings_users.shape[0]):
  for j in range(estimated_ratings_users.shape[1]):
    estimated_ratings_users[i][j] = 0           

In [ ]:
# Validação para método baseado em usuários, sem considerar as ocupações
start_time = time.time()
for sim in sim_type:
  for k in k_values:
    mean_mse = 0
    for i in random_index:
      # Cálculo de todas as similaridades para o usuário i
      for j in range(6040):
        if (j != i):
          sim_users[i][j] = similarity(X=matrix_cos_users[i,0:3885],Y=matrix_cos_users[j,0:3885],type=sim)
        else:
          sim_users[i][j] = 0
      
      evaluated_index = []
      for m in range(3883):
        if (matrix_cos_users[i][m] != 0):
          evaluated_index.append(m)      

      X_target = []
      X_estimated = []
      # Cálculo das estimativas de nota para o usuário i
      for j in evaluated_index:
        estimated_ratings_users[i][j] = estimated_rating(sim_vec=sim_users[i],ratings_column=matrix_cos_users[:,j],k=k)
        if (estimated_ratings_users[i][j] > 0):
          X_target.append(matrix_cos_users[i][j])
          X_estimated.append(estimated_ratings_users[i][j])

      X_target = np.array(X_target)
      X_estimated = np.array(X_estimated)
      mean_mse += mean_squared_error(X_target=X_target,X_estimated=X_estimated)
    mean_mse = mean_mse/len(random_index)
    mse_users_dict_2[mean_mse] = [sim,k]

print('Tempo de execução:',time.time()-start_time,'segundos')
print('Melhores hiperparâmetros:',mse_users_dict_2[min(list(mse_users_dict_2.keys()))],'-> MSE:',min(list(mse_users_dict_2.keys())))

Tempo de execução: 297.16802859306335 segundos
Melhores hiperparâmetros: ['cosine', 45] -> MSE: 0.8693848917409894


In [ ]:
# Limpeza das matrizes de similaridade e de estimativas
for i in range(sim_users.shape[0]):
  for j in range(sim_users.shape[1]):
    sim_users[i][j] = 0

for i in range(estimated_ratings_users.shape[0]):
  for j in range(estimated_ratings_users.shape[1]):
    estimated_ratings_users[i][j] = 0           

In [ ]:
# Validação para método baseado em usuários, sem considerar as ocupações, idade e sexo
start_time = time.time()
for sim in sim_type:
  for k in k_values:
    mean_mse = 0
    for i in random_index:
      # Cálculo de todas as similaridades para o usuário i
      for j in range(6040):
        if (j != i):
          sim_users[i][j] = similarity(X=matrix_cos_users[i,0:3883],Y=matrix_cos_users[j,0:3883],type=sim)
        else:
          sim_users[i][j] = 0
      
      evaluated_index = []
      for m in range(3883):
        if (matrix_cos_users[i][m] != 0):
          evaluated_index.append(m)      

      X_target = []
      X_estimated = []
      # Cálculo das estimativas de nota para o usuário i
      for j in evaluated_index:
        estimated_ratings_users[i][j] = estimated_rating(sim_vec=sim_users[i],ratings_column=matrix_cos_users[:,j],k=k)
        if (estimated_ratings_users[i][j] > 0):
          X_target.append(matrix_cos_users[i][j])
          X_estimated.append(estimated_ratings_users[i][j])

      X_target = np.array(X_target)
      X_estimated = np.array(X_estimated)
      mean_mse += mean_squared_error(X_target=X_target,X_estimated=X_estimated)
    mean_mse = mean_mse/len(random_index)
    mse_users_dict_3[mean_mse] = [sim,k]

print('Tempo de execução:',time.time()-start_time,'segundos')
print('Melhores hiperparâmetros:',mse_users_dict_3[min(list(mse_users_dict_3.keys()))],'-> MSE:',min(list(mse_users_dict_3.keys())))

Tempo de execução: 297.25329971313477 segundos
Melhores hiperparâmetros: ['cosine', 45] -> MSE: 0.8671792785025716


In [ ]:
# Criação de um vetor com os índices referentes a 20 filmes aleatórios
random_index = np.random.randint(low=0,high=3883,size=20)

In [ ]:
i_aux = 0      # Índice auxiliar que conta quantos filmes não possuem avaliação dentre aqueles selecionados para o cálculo do MSE

k_values = np.linspace(start=46,stop=55,num=10,dtype=int)                        # Lista com todos os valores de k

# Validação para método baseado em filmes, considerando os gêneros
start_time = time.time()
for sim in sim_type:
  for k in k_values:
    mean_mse = 0
    i_aux = 0
    for i in random_index:
      # Cálculo de todas as similaridades para o filme i
      for j in range(3883):
        if (j != i):
          sim_movies[i][j] = similarity(X=matrix_cos_movies[i],Y=matrix_cos_movies[j],type=sim)
        else:
          sim_movies[i][j] = 0
      
      evaluated_index = []
      for m in range(6040):
        if (matrix_cos_movies[i][m] != 0):
          evaluated_index.append(m)      

      X_target = []
      X_estimated = []
      # Cálculo das estimativas de nota para o filme i, pelos usuários
      for j in evaluated_index:
        estimated_ratings_movies[i][j] = estimated_rating(sim_vec=sim_movies[i],ratings_column=matrix_cos_movies[:,j],k=k)
        if (estimated_ratings_movies[i][j] > 0):
          X_target.append(matrix_cos_movies[i][j])
          X_estimated.append(estimated_ratings_movies[i][j])

      X_target = np.array(X_target)
      X_estimated = np.array(X_estimated)
      try:
        mean_mse += mean_squared_error(X_target=X_target,X_estimated=X_estimated)
      except ZeroDivisionError:
        i_aux += 1
    mean_mse = mean_mse/(len(random_index)-i_aux)
    mse_movies_dict_1[mean_mse] = [sim,k]

print('Tempo de execução:',time.time()-start_time,'segundos')
print('Melhores hiperparâmetros:',mse_movies_dict_1[min(list(mse_movies_dict_1.keys()))],'-> MSE:',min(list(mse_movies_dict_1.keys())))

Tempo de execução: 215.1876859664917 segundos
Melhores hiperparâmetros: ['cosine', 46] -> MSE: 1.0011902638629626


In [ ]:
# Limpeza das matrizes de similaridade e de estimativas
for i in range(sim_movies.shape[0]):
  for j in range(sim_movies.shape[1]):
    sim_movies[i][j] = 0

for i in range(estimated_ratings_movies.shape[0]):
  for j in range(estimated_ratings_movies.shape[1]):
    estimated_ratings_movies[i][j] = 0          

In [ ]:
i_aux = 0      # Índice auxiliar que conta quantos filmes não possuem avaliação dentre aqueles selecionados para o cálculo do MSE

# Validação para método baseado em filmes, sem considerar os gêneros
start_time = time.time()
for sim in sim_type:
  for k in k_values:
    mean_mse = 0
    i_aux = 0
    for i in random_index:
      # Cálculo de todas as similaridades para o filme i
      for j in range(3883):
        if (j != i):
          sim_movies[i][j] = similarity(X=matrix_cos_movies[i,0:6040],Y=matrix_cos_movies[j,0:6040],type=sim)
        else:
          sim_movies[i][j] = 0
      
      evaluated_index = []
      for m in range(6040):
        if (matrix_cos_movies[i][m] != 0):
          evaluated_index.append(m)      

      X_target = []
      X_estimated = []
      # Cálculo das estimativas de nota para o filme i, pelos usuários
      for j in evaluated_index:
        estimated_ratings_movies[i][j] = estimated_rating(sim_vec=sim_movies[i],ratings_column=matrix_cos_movies[:,j],k=k)
        if (estimated_ratings_movies[i][j] > 0):
          X_target.append(matrix_cos_movies[i][j])
          X_estimated.append(estimated_ratings_movies[i][j])

      X_target = np.array(X_target)
      X_estimated = np.array(X_estimated)
      try:
        mean_mse += mean_squared_error(X_target=X_target,X_estimated=X_estimated)
      except ZeroDivisionError:
        i_aux += 1
    mean_mse = mean_mse/(len(random_index)-i_aux)
    mse_movies_dict_2[mean_mse] = [sim,k]

print('Tempo de execução:',time.time()-start_time,'segundos')
print('Melhores hiperparâmetros:',mse_movies_dict_2[min(list(mse_movies_dict_2.keys()))],'-> MSE:',min(list(mse_movies_dict_2.keys())))

Tempo de execução: 212.62211871147156 segundos
Melhores hiperparâmetros: ['cosine', 46] -> MSE: 0.998212270137023


In [ ]:
# Escolha de um usuário aleatório
user_index = np.random.randint(low=0,high=6040,size=1)[0]

# Lista com as notas dadas pelo usuário
ratings_user = []
# Lista com os índices dos filmes avaliados pelo usuário
movies_index = []

for j in range(3883):
  if (matrix_cos_users[user_index][j] != 0):
    ratings_user.append(matrix_cos_users[user_index][j])
    movies_index.append(j)

# Estrutura de dados criada para o usuário
df_user = pd.DataFrame()
df_user[0] = np.array(ratings_user)
df_user[1] = np.array(movies_index)
df_user.columns = ['Ratings','Movies indexes']

# Ordenação das avaliações do usuário em ordem decrescente
df_user = df_user.sort_values(by='Ratings',ascending=False)

# Lista com os índices correspondentes aos 10 melhores filmes, de acordo com o usuário aleatório
movies_best_index = list(df_user['Movies indexes'][0:10])

# Criação de dicionário que contém os gêneros cinematográficos presentes nos 10 melhores filmes, de acordo com o usuário aleatório
genres_dict = {}

for movie_ind in movies_best_index:
  for j in range(18):
    if (movies_new[movies_new.columns[2+j]][movie_ind] != 0):
      if (movies_new.columns[2+j] in genres_dict.keys()):
        genres_dict[movies_new.columns[2+j]] += 1
      else:
        genres_dict[movies_new.columns[2+j]] = 1
        
print('Gêneros presentes nos 10 melhores filmes, para o usuário',user_index+1,':\n')

for item in genres_dict.items():
  print(item)

Gêneros presentes nos 10 melhores filmes, para o usuário 323 :

('Adventure', 2)
("Children's", 1)
('Fantasy', 1)
('Action', 4)
('Thriller', 3)
('Sci-Fi', 4)
('Musical', 2)
('Romance', 3)
('Crime', 1)
('Mystery', 2)
('Film-Noir', 1)
('Drama', 2)
('War', 2)


In [ ]:
# Cálculo das similaridades entre o usuário aleatório e os demais
for j in range(6040):
  if (j != i):
    sim_users[user_index][j] = similarity(X=matrix_cos_users[user_index,0:3883],Y=matrix_cos_users[j,0:3883],type='cosine')
  else:
    sim_users[user_index][j] = 0

# Criação de um dicionário que faz a correspondência entre notas estimadas e os índices dos filmes (índice não é o mesmo que ID)
ratings_index_dict = {}

ratings_best = []

# Cálculos das estimativas para os filmes ainda não avaliados pelo usuário
for j in range(3883):
  if (j in movies_index):
    continue
  else:
    estimated_ratings_users[user_index][j] = estimated_rating(sim_vec=sim_users[user_index],ratings_column=matrix_cos_users[:,j],k=45)
    ratings_index_dict[estimated_ratings_users[user_index][j]] = j
    ratings_best.append(estimated_ratings_users[user_index][j])

# Ordenação das 10 melhores notas estimadas
ratings_best.sort(reverse=True)

print('10 melhores recomendações para o usuário',user_index+1,':\n')

for i in range(10):
  genres = []
  index = ratings_index_dict[ratings_best[i]]
  print('Título:',movies_new['Title'][index])
  for j in range(18):
    if (movies_new[movies_new.columns[2+j]][index] == 1):
      genres.append(movies_new.columns[2+j])
  print('Gêneros:',genres)
  print('Nota estimada:',ratings_best[i])
  print('\n')

10 melhores recomendações para o usuário 323 :

Título: Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954)
Gêneros: ['Drama', 'Action']
Nota estimada: 4.736212188820336


Título: Godfather, The (1972)
Gêneros: ['Drama', 'Action', 'Crime']
Nota estimada: 4.734683926810816


Título: Godfather: Part II, The (1974)
Gêneros: ['Drama', 'Action', 'Crime']
Nota estimada: 4.661015026395038


Título: Pulp Fiction (1994)
Gêneros: ['Drama', 'Crime']
Nota estimada: 4.646584171956733


Título: Schindler's List (1993)
Gêneros: ['Drama', 'War']
Nota estimada: 4.639395250462517


Título: Shawshank Redemption, The (1994)
Gêneros: ['Drama']
Nota estimada: 4.6015050364555075


Título: Raiders of the Lost Ark (1981)
Gêneros: ['Action', 'Adventure']
Nota estimada: 4.598966861977359


Título: For All Mankind (1989)
Gêneros: ['Documentary']
Nota estimada: 4.541096283453555


Título: Wrong Trousers, The (1993)
Gêneros: ['Animation', 'Comedy']
Nota estimada: 4.540187293724761


Título: Sanjuro 